In [1]:
import codecs

In [2]:
import sys
import nltk
from nltk.stem.porter import *
from sklearn.feature_extraction import stop_words
import xml.etree.cElementTree as ET
from collections import Counter
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import zipfile
import os
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# from wordcloud import WordCloud
PARTIALS = False

/home/users/ssingla2/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
file = open("../uncertainty_words.txt", "r+")
uncertainty_str = file.read()
uncertainty_list = uncertainty_str.lower().split(",")
len(uncertainty_list)

297

In [4]:
path = '/ifs/gsb/usf_interns/Parser_Project/ParsedDocumentsFolder/10KParsed/Regression_10Ks/'
files=os.listdir(path)

In [5]:
files[2550]

'parsed_10-K_0001193125-14-078778.txt'

In [6]:
corpus=dict()
for file in files:
    text = codecs.open(path+file, 'r', encoding='utf-8', errors='ignore').read()
    corpus.update({file:text})

In [7]:
def tokenizer(text):
    """
    Tokenize text and return a non-unique list of tokenized words
    found in the text. Normalize to lowercase, strip punctuation,
    remove stop words, drop words of length < 3, strip digits.
    """
    tokenizer = RegexpTokenizer(r'\w+')
    valid_characters = string.printable
    text = ''.join(i for i in text if i in valid_characters)
    text = text.lower()
    text = re.sub('[' + string.punctuation + '0-9\\r\\t\\n\®]', ' ', text)
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if len(w) > 2]  # ignore a, an, to, at, be, ...
    goodWords = [w for w in tokens if w not in ENGLISH_STOP_WORDS]
    return(goodWords)

In [8]:
def compute_tfidf(corpus):
    """
    Create and return a TfidfVectorizer object after training it on
    the list of articles pulled from the corpus dictionary. Meaning,
    call fit() on the list of document strings, which figures out
    all the inverse document frequencies (IDF) for use later by
    the transform() function. The corpus argument is a dictionary 
    mapping file name to xml text.
    """

    tfidf = TfidfVectorizer(input='content',
                        analyzer='word',
                        tokenizer=tokenizer,
                        stop_words='english',
                        decode_error = 'ignore')
    list_of_articles=[]
    for key in corpus:
        list_of_articles.append(corpus[key])      
    tfidf_matrix=tfidf.fit(list_of_articles)  
    return tfidf_matrix 

In [9]:
tfidf=compute_tfidf(corpus)

/home/users/ssingla2/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def summarize(tfidf, text):
    """
    Given a trained TfidfVectorizer object and some XML text, return
    up to n (word,score) pairs in a list. Discard any terms with
    scores < 0.09.
    """
    tmatrix=tfidf.transform([text])
    retTuple=[]
    feature_index=tmatrix.nonzero()[1]
    feature_words=tfidf.get_feature_names()
    for i in feature_index:
        retTuple.append((feature_words[i],tmatrix[0,i]))

    def Sort_Tuple(tup):  
        return(sorted(tup, key = lambda x: x[1],reverse = True))
    
    sorted_list=Sort_Tuple(retTuple)
#     sortedlistn=(sorted_list[:n])
#     sortedlistnFinal=[i for i in sortedlistn if i[1] >= 0.09]
    return(sorted_list)

In [11]:
outputTfidfEachDict={}
outputTfidfTotalDict={}
for key in files:
    Text_10K=corpus[key]
    tfidf_scores=summarize(tfidf, Text_10K)
    dict_10={}
    for item in tfidf_scores:
        dict_10.update( {item[0] :item[1]} )
    uncertainty_dict={}
    total_tfidf_unc=0
    for word in uncertainty_list:
        if word in dict_10:
            uncertainty_dict[word]=dict_10[word]
            total_tfidf_unc=total_tfidf_unc+dict_10[word] 
    outputTfidfEachDict[key] = uncertainty_dict
    outputTfidfTotalDict[key]= total_tfidf_unc
            

In [13]:
with open('saved_uncertainty_tfidf_dicts.txt', 'w') as f:
    f.write(str(outputTfidfEachDict))

In [16]:
%store outputTfidfTotalDict

Stored 'outputTfidfTotalDict' (dict)


In [14]:
outputTfidfTotalDict

{'parsed_10-K_0000882835-13-000008.txt': 0.3535839006733593,
 'parsed_10-K_0001193125-10-053838.txt': 0.270995693421881,
 'parsed_10-K_0000320187-13-000092.txt': 0.36116028094702235,
 'parsed_10-K_0001095073-14-000008.txt': 0.35437008850120943,
 'parsed_10-K_0000004904-13-000025.txt': 0.14890800618880223,
 'parsed_10-K_0001308179-13-000024.txt': 0.1956771354859622,
 'parsed_10-K_0000950123-11-018684.txt': 0.25431168547812905,
 'parsed_10-K_0001193125-12-078626.txt': 0.35312887227552664,
 'parsed_10-K_0000874761-15-000009.txt': 0.3120602600534997,
 'parsed_10-K_0001521332-14-000011.txt': 0.25519072799362347,
 'parsed_10-K_0001090872-13-000029.txt': 0.3443155263585258,
 'parsed_10-K_0001358071-14-000005.txt': 0.3234988997402449,
 'parsed_10-K_0000078239-11-000015.txt': 0.11678707751366547,
 'parsed_10-K_0000021344-14-000008.txt': 0.24047364692942882,
 'parsed_10-K_0000898173-13-000006.txt': 0.27407227638222903,
 'parsed_10-K_0001047469-12-001182.txt': 0.2512365371417208,
 'parsed_10-K_00

Code for Single file:

In [ ]:
key='parsed_10-K_0000006201-13-000023.txt'
Text_10K=corpus[key]
tfidf_scores=summarize(tfidf, Text_10K)
dict_10={}
for item in tfidf_scores:
    dict_10.update( {item[0] :item[1]} )
    

In [ ]:
uncertainty_dict={}
total_tfidf_unc=0
for word in uncertainty_list:
    if word in dict_10:
        uncertainty_dict[word]=dict_10[word]
        total_tfidf_unc=total_tfidf_unc+dict_10[word]

In [ ]:
total_tfidf_unc

In [ ]:
uncertainty_dict